## Import necessary libraries

In [1]:
import pickle
import sys
import timeit
import os

import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from statsmodels.stats.contingency_tables import mcnemar

#### Check if CUDA is available

In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

## Define NN class

In [3]:
class MolecularPropertyPrediction(nn.Module):
    
    def __init__(self):
        super(MolecularPropertyPrediction, self).__init__()
        self.embed_atom = nn.Embedding(n_fingerprint, dim)
        self.W_atom = nn.ModuleList([nn.Linear(dim, dim) for _ in range(layer)])
        self.W_property = nn.Linear(dim+extra_dim, 11)
    
    """Pad adjacency matrices for batch processing."""
    def pad(self, matrices, value):
        sizes = [d.shape[0] for d in matrices]
        D = sum(sizes)
        pad_matrices = value + np.zeros((D, D))
        m = 0
        for i, d in enumerate(matrices):
            s_i = sizes[i]
            pad_matrices[m:m+s_i, m:m+s_i] = d
            m += s_i
        return torch.FloatTensor(pad_matrices).to(device)
    
    def sum_axis(self, xs, axis):
        y = list(map(lambda x: torch.sum(x, 0), torch.split(xs, axis)))
        return torch.stack(y)
    
    def update(self, xs, adjacency, i):
        hs = torch.relu(self.W_atom[i](xs))
        return torch.matmul(adjacency, hs)
    
    def forward(self, inputs, sel_maccs):
        
        atoms, adjacency = inputs
        axis = list(map(lambda x: len(x), atoms))

        atoms = torch.cat(atoms)
        x_atoms = self.embed_atom(atoms)
        adjacency = self.pad(adjacency, 0)

        for i in range(layer):
            x_atoms = self.update(x_atoms, adjacency, i)
        
        extra_inputs = sel_maccs.to(device)
        y_molecules = self.sum_axis(x_atoms, axis)
        
        #AM: Enable this if we want extra_inputs.
        y_molecules = torch.cat((y_molecules,extra_inputs),1)
        z_properties = self.W_property(y_molecules)
        
        return z_properties
    
    def embed_inputs(self, inputs, sel_maccs):
        atoms, adjacency = inputs
        axis = list(map(lambda x: len(x), atoms))

        atoms = torch.cat(atoms)
        x_atoms = self.embed_atom(atoms)
        adjacency = self.pad(adjacency, 0)

        for i in range(layer):
            x_atoms = self.update(x_atoms, adjacency, i)
        
        extra_inputs = sel_maccs.to(device)
        y_molecules = self.sum_axis(x_atoms, axis)
        
        y_molecules = torch.cat((y_molecules,extra_inputs),1)
        return(y_molecules)    
    
    
    def __call__(self, data_batch, train=True, correlation_analysis=False):
        
        sel_maccs = torch.FloatTensor(data_batch[-1])
        
        inputs, t_properties = data_batch[:-2], torch.cat(data_batch[-2])
        
        if correlation_analysis:
            embedding = self.embed_inputs(inputs, sel_maccs)
            return(embedding)
        
        z_properties = self.forward(inputs, sel_maccs)
        
        if train:
            loss = F.cross_entropy(z_properties, t_properties)
            return loss
        else:
            zs = F.softmax(z_properties, 1).to('cpu').data.numpy()
            ts = t_properties.to('cpu').data.numpy()
            scores = list(map(lambda x: x.argsort()[-3:][::-1], zs))
            labels = list(map(lambda x: np.argmax(x), zs))
            return scores, labels, ts

#### Create trainer class

In [4]:
class Trainer(object):
    
    def __init__(self, model):
        self.model = model
        self.optimizer = optim.Adam(self.model.parameters(), lr=lr)
        
    def train(self, dataset_train):
        np.random.shuffle(dataset_train)
        N = len(dataset_train)
        loss_total = 0
        for i in range(0, N, batch):
            data_batch = list(zip(*dataset_train[i:i+batch]))
            loss = self.model(data_batch)
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            loss_total += loss.to('cpu').data.numpy()
        return loss_total

#### Create tester class

In [5]:
class Tester(object):
    
    def __init__(self, model):
        self.model = model

    def test(self, dataset_test):

        N = len(dataset_test)
        score_list, label_list, t_list = [], [], []

        for i in range(0, N, batch):
            data_batch = list(zip(*dataset_test[i:i+batch]))
            scores, labels, ts = self.model(data_batch, train=False)
            score_list = np.append(score_list, scores)
            label_list = np.append(label_list, labels)
            t_list = np.append(t_list, ts)
        
        neg_pos_list1 = []
        neg_pos_list2 = []
        neg_pos_list3 = []
        auc = accuracy_score(t_list, label_list)
        
        auc2, auc3 = 0, 0 
        for i in range(0, N):
            true_class = t_list[i]
            
            if true_class == score_list[3*i]:
                neg_pos_list1.append(1)
            else:
                neg_pos_list1.append(0)
            
            if true_class in score_list[3*i:3*i+2]:
                auc2 += 1
                neg_pos_list2.append(1)
            else:
                neg_pos_list2.append(0)
                
            if true_class in score_list[3*i:3*i+3]:
                auc3 += 1
                neg_pos_list3.append(1)
            else:
                neg_pos_list3.append(0)

        return auc, auc2/N, auc3/N, neg_pos_list1, neg_pos_list2, neg_pos_list3
    
    def result(self, epoch, time, loss, auc_dev,
               auc_test, file_result):
        with open(file_result, 'a') as f:
            result = map(str, [epoch, time, loss, auc_dev, auc_test])
            f.write('\t'.join(result) + '\n')

    def save_model(self, model, file_name):
        torch.save(model, file_name)

### Helper functions

In [6]:
def load_tensor(file_name, dtype):
    return [dtype(d).to(device) for d in np.load(file_name + '.npy', allow_pickle=True)]


def load_numpy(file_name):
    return np.load(file_name + '.npy', allow_pickle=True)


def load_pickle(file_name):
    with open(file_name, 'rb') as f:
        return pickle.load(f)


def shuffle_dataset(dataset, seed):
    np.random.seed(seed)
    np.random.shuffle(dataset)
    return dataset


def split_dataset(dataset, ratio):
    n = int(ratio * len(dataset))
    dataset_1, dataset_2 = dataset[:n], dataset[n:]
    return dataset_1, dataset_2

#### Hyperparameters

In [7]:
radius         = 2
dim            = 50
layer          = 2
batch          = 10
lr             = 1e-3
lr_decay       = 0.5
decay_interval = 10
iteration      = 100
extra_dim      = 20

(dim, layer, batch, decay_interval, iteration, extra_dim) = map(int, [dim, layer, batch, decay_interval, iteration, extra_dim])
lr, lr_decay = map(float, [lr, lr_decay])

### Create and train model

In [11]:
dir_input = ('inputgcn'+str(radius)+'/')

molecules    = load_tensor(dir_input + 'molecules', torch.LongTensor)
adjacencies  = load_numpy(dir_input + 'adjacencies')
t_properties = load_tensor(dir_input + 'properties', torch.LongTensor)
maccs        = load_numpy(dir_input + 'maccs')

with open(dir_input + 'fingerprint_dict.pickle', 'rb') as f:
    fingerprint_dict = pickle.load(f)
    unknown          = 100
    n_fingerprint    = len(fingerprint_dict) + unknown
    
dataset = list(zip(molecules, adjacencies, t_properties, maccs))
dataset = shuffle_dataset(dataset, 1234)
dataset_train, dataset_   = split_dataset(dataset, 0.8)
dataset_dev, dataset_test = split_dataset(dataset_, 0.5)

torch.manual_seed(1234)

model   = MolecularPropertyPrediction().to(device)
trainer = Trainer(model)
tester  = Tester(model)

dir_output = ('output/')
os.makedirs(dir_output, exist_ok=True)

dir_result = ('output/result/')
os.makedirs(dir_result, exist_ok=True)

dir_model  = ('output/model/')
os.makedirs(dir_model, exist_ok=True)

file_result = dir_result + '.txt'
with open(file_result, 'w') as f:
    f.write('Epoch\tTime(sec)\tLoss_train\tAUC_dev\tAUC_test\n')

file_model = dir_model + 'model'

In [12]:
print('Training...')
print('Epoch \t Time(sec) \t Loss_train \t AUC_dev \t AUC2_dev \t AUC3_dev \t AUC_test \t AUC2_test \t AUC3_test')

start = timeit.default_timer()

for epoch in range(iteration):
    if (epoch+1) % decay_interval == 0:
        trainer.optimizer.param_groups[0]['lr'] *= lr_decay

    loss = trainer.train(dataset_train)
    auc_dev, auc2_dev, auc3_dev, _, _, _ = tester.test(dataset_dev)
    auc_test, auc2_test, auc3_test, gcn_nplist1, gcn_nplist2, gcn_nplist3 = tester.test(dataset_test)
    
    lr_rate = trainer.optimizer.param_groups[0]['lr']

    end = timeit.default_timer()
    time = end - start

    tester.result(epoch, time, loss, auc_dev, auc_test, file_result)
    tester.save_model(model, file_model)

    print('%d \t %.4f \t %.4f \t %.4f \t %.4f \t %.4f \t %.4f \t %.4f \t %.4f' %(epoch, time, loss, auc_dev, auc2_dev, auc3_dev, auc_test, auc2_test, auc3_test))

Training...
Epoch 	 Time(sec) 	 Loss_train 	 AUC_dev 	 AUC2_dev 	 AUC3_dev 	 AUC_test 	 AUC2_test 	 AUC3_test


C:\Users\mayan\AppData\Local\Temp\ipykernel_18628\647757525.py:73: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:210.)
  sel_maccs = torch.FloatTensor(data_batch[-1])


0 	 8.5518 	 538.8387 	 0.5947 	 0.7357 	 0.8194 	 0.6872 	 0.7930 	 0.8590
1 	 9.9744 	 335.4248 	 0.6762 	 0.7996 	 0.8789 	 0.7423 	 0.8480 	 0.9317
2 	 11.4728 	 265.0929 	 0.7467 	 0.8568 	 0.8987 	 0.7863 	 0.8965 	 0.9383
3 	 12.9603 	 211.1913 	 0.7357 	 0.8612 	 0.9075 	 0.7731 	 0.8811 	 0.9405
4 	 14.6474 	 172.1952 	 0.7974 	 0.8767 	 0.9229 	 0.8370 	 0.9273 	 0.9648
5 	 16.4042 	 145.3180 	 0.8084 	 0.8833 	 0.9207 	 0.8282 	 0.9295 	 0.9626
6 	 18.0658 	 125.2414 	 0.8172 	 0.8965 	 0.9251 	 0.8370 	 0.9449 	 0.9714
7 	 19.7409 	 110.7273 	 0.8062 	 0.8899 	 0.9273 	 0.8502 	 0.9405 	 0.9670
8 	 21.4575 	 91.8339 	 0.8304 	 0.9009 	 0.9295 	 0.8590 	 0.9471 	 0.9670
9 	 23.1285 	 67.8698 	 0.8414 	 0.8987 	 0.9295 	 0.8700 	 0.9493 	 0.9736
10 	 24.9020 	 61.4977 	 0.8238 	 0.9009 	 0.9405 	 0.8700 	 0.9471 	 0.9692
11 	 26.8341 	 57.9448 	 0.8128 	 0.8987 	 0.9207 	 0.8546 	 0.9251 	 0.9692
12 	 28.8838 	 55.7639 	 0.8348 	 0.9053 	 0.9383 	 0.8833 	 0.9493 	 0.9736
13 

# Let's use Random Forest

In [14]:
dir_input = ('input'+str(radius)+'/')

molecules    = load_tensor(dir_input + 'molecules', torch.LongTensor)
adjacencies  = load_numpy(dir_input + 'adjacencies')
t_properties = load_tensor(dir_input + 'properties', torch.LongTensor)
maccs        = load_numpy(dir_input + 'maccs')

with open(dir_input + 'fingerprint_dict.pickle', 'rb') as f:
    fingerprint_dict = pickle.load(f)
    
dataset = list(zip(molecules, adjacencies, t_properties, maccs))
dataset = shuffle_dataset(dataset, 1234)
dataset_train, dataset_   = split_dataset(dataset, 0.8)
dataset_dev, dataset_test = split_dataset(dataset_, 0.5)

data_batch = list(zip(*dataset_train))
properties_train, maccs_train = data_batch[-2], data_batch[-1]

data_batch = list(zip(*dataset_dev))
properties_dev, maccs_dev = data_batch[-2], data_batch[-1]

data_batch = list(zip(*dataset_test))
properties_test, maccs_test = data_batch[-2], data_batch[-1]

train_len, dev_len, test_len = len(dataset_train), len(dataset_dev), len(dataset_test)

feature_len = maccs_train[0].shape[0]

X_train, X_dev, X_test = np.zeros((train_len,feature_len)), np.zeros((dev_len,feature_len)), np.zeros((test_len,feature_len))
Y_train, Y_dev, Y_test = np.zeros((train_len,)), np.zeros((dev_len,)), np.zeros((test_len,))

for i in range(train_len):
    X_train[i,:] = maccs_train[i]
    Y_train[i] = properties_train[i][0]
    
for i in range(dev_len):
    X_dev[i,:]   = maccs_dev[i]
    Y_dev[i]   = properties_dev[i][0]
    
for i in range(test_len):
    X_test[i,:]  = maccs_test[i]
    Y_test[i]  = properties_test[i][0]

fingerprint_dict = load_pickle(dir_input + 'fingerprint_dict.pickle')
unknown          = 100
n_fingerprint    = len(fingerprint_dict) + unknown

In [15]:
clf = RandomForestClassifier(n_estimators=300, criterion = 'gini', max_depth=60, random_state=0)
clf.fit(X_train, Y_train)

rf_neg_pos_list1 = []
rf_neg_pos_list2 = []
rf_neg_pos_list3 = []

Y_pred = clf.predict_proba(X_test)

num_classes = 3
top_n = np.argsort(Y_pred)[:,:-num_classes-1:-1]
top_n = clf.classes_[top_n]

TP1 = 0
TP2 = 0
TP3 = 0

FN1 = 0
FN2 = 0
FN3 = 0

for i in range(X_test.shape[0]):
    true_class = Y_test[i]
    
    if true_class in top_n[i,0:1]:
        TP1 += 1
        rf_neg_pos_list1.append(1)
    else:
        FN1 += 1
        rf_neg_pos_list1.append(0)
    
    if true_class in top_n[i,0:2]:
        TP2 += 1
        rf_neg_pos_list2.append(1)
    else:
        FN2 += 1
        rf_neg_pos_list2.append(0)
        
    if true_class in top_n[i,0:3]:
        TP3 += 1
        rf_neg_pos_list3.append(1)
    else:
        FN3 += 1
        rf_neg_pos_list3.append(0)
print('Total samples : %d, Top-1 Correct : %d, Top-1 Incorrect : %d' %(TP1+FN1, TP1, FN1))
print('Prediction Accuracy Top-1 : %.2f%%' %(100.0*TP1/(TP1+FN1)))
        
print('Total samples : %d, Top-2 Correct : %d, Top-2 Incorrect : %d' %(TP2+FN2, TP2, FN2))
print('Prediction Accuracy Top-2 : %.2f%%' %(100.0*TP2/(TP2+FN2)))

print('Total samples : %d, Top-3 Correct : %d, Top-3 Incorrect : %d' %(TP3+FN3, TP3, FN3))
print('Prediction Accuracy Top-3 : %.2f%%' %(100.0*TP3/(TP3+FN3)))

Total samples : 454, Top-1 Correct : 398, Top-1 Incorrect : 56
Prediction Accuracy Top-1 : 87.67%
Total samples : 454, Top-2 Correct : 433, Top-2 Incorrect : 21
Prediction Accuracy Top-2 : 95.37%
Total samples : 454, Top-3 Correct : 440, Top-3 Incorrect : 14
Prediction Accuracy Top-3 : 96.92%
